<a href="https://colab.research.google.com/github/kushalvanama/INFO5731_Spring2020/blob/main/Vanama_In_class_exercise_06.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **The sixth in-class-exercise (20 points in total, 3/2/2021)**

## **1. Rule-based information extraction (10 points)**

Use any keywords related to data science, natural language processing, machine learning to search from google scholar, get the **titles** of 100 articles (either by web scraping or manually) about this topic, define a set of patterns to extract the research questions/problems, methods/algorithms/models, datasets, applications, or any other important information about this topic. 

In [1]:
# Write your code here

from IPython.core.display import display, HTML
display(HTML('<style>.container { width:80% !important; }</style>'))
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np


headers = [{'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:81.0) Gecko/20100101 Firefox/81.0'},
           {'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.75 Safari/537.36 Vivaldi/3.3'},
           {'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.75 Safari/537.36'},
           {'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.75 Safari/537.36 Edg/86.0.622.38'}]

pages = np.arange(1, 11) * 10 

responses = []
for page in pages:
    url = 'https://scholar.google.com/scholar?start=' + str(page) + '&q=natural+language+processing&hl=en&as_sdt=0,44'
    print(f'{page} titles being extracted')
    random_index = np.random.randint(0, 4)
    header = headers[random_index]
    response = requests.get(url, headers=header)
    if response.status_code == 200:
        print('Everything is fine. Continue to scrap ...')
    else:
        print('Something went wrong!')
    soup=BeautifulSoup(response.content,'lxml')
    responses.append(soup)


10 titles being extracted
Everything is fine. Continue to scrap ...
20 titles being extracted
Everything is fine. Continue to scrap ...
30 titles being extracted
Everything is fine. Continue to scrap ...
40 titles being extracted
Everything is fine. Continue to scrap ...
50 titles being extracted
Everything is fine. Continue to scrap ...
60 titles being extracted
Everything is fine. Continue to scrap ...
70 titles being extracted
Everything is fine. Continue to scrap ...
80 titles being extracted
Everything is fine. Continue to scrap ...
90 titles being extracted
Everything is fine. Continue to scrap ...
100 titles being extracted
Everything is fine. Continue to scrap ...


In [2]:
titles = []
count = 1
for response in responses:
    for item in response.select('[data-lid]'):
        try:
            titles.append(item.select('h3')[0].get_text())
        except Exception as error:
            print('Something went wrong!')
        count += 1

In [3]:
df = pd.DataFrame({'titles' : titles})
df.head()

,titles
0,Natural language processing
1,[BOOK][B] Natural language processing
2,[PDF][PDF] Natural language processing
3,[BOOK][B] Strategies for natural language proc...
4,[PDF][PDF] Human engineering fcr applied natur...


In [5]:
df.to_csv('titles.csv')

**DATA PRE-PROCESSING**

In [6]:
import pandas as pd
import numpy as np
import spacy
nlp = spacy.load('en_core_web_sm')
pd.set_option('display.max_colwidth', 200)

df = pd.read_csv('titles.csv')
df.titles.values

array(['Natural language processing',
       '[BOOK][B] Natural language processing',
       '[PDF][PDF] Natural language processing',
       '[BOOK][B] Strategies for natural language processing',
       '[PDF][PDF] Human engineering fcr applied natural language processing',
       'Practical Natural-Language Processing by Computer',
       'A primer on neural network models for natural language processing',
       '[BOOK][B] Introduction to natural language processing',
       'Allennlp: A deep semantic natural language processing platform',
       'Introduction to Arabic natural language processing',
       'A maximum entropy approach to natural language processing',
       '[PDF][PDF] Natural language processing',
       '[BOOK][B] The handbook of computational linguistics and natural language processing',
       "HuggingFace's Transformers: State-of-the-art natural language processing",
       '[PDF][PDF] Natural language processing (almost) from scratch',
       'An overview of e

In [7]:
def clean_title(s):
    discards = ['[PDF][PDF] ', '[BOOK][B] ', '[HTML][HTML] ', '[CITATION][C] ']
    for discard in discards:
        if discard in s:
            s = s.replace(discard, '')
    return s
df.titles = df.titles.apply(clean_title)
df.titles

0                                                                                            Natural language processing
1                                                                                            Natural language processing
2                                                                                            Natural language processing
3                                                                             Strategies for natural language processing
4                                                              Human engineering fcr applied natural language processing
                                                             ...                                                        
95                                                            Similarity-based approaches to natural language processing
96                                                   Opportunities for natural language processing research in education
97            Sentiment analyzer

In [8]:
def clean_text(text):
    import re
    # removing apostrophes
    text = re.sub("'s",'',str(text))
    # removing hyphens
    text = re.sub("-",' ',str(text))
    text = re.sub("— ",'',str(text))
    # removing quotation marks
    text = re.sub('\"','',str(text))
    # removing any reference to outside text
    text = re.sub('[^\w\s]','', str(text))
    return text

df.titles = df.titles.apply(clean_text)
df.titles

0                                                                                           Natural language processing
1                                                                                           Natural language processing
2                                                                                           Natural language processing
3                                                                            Strategies for natural language processing
4                                                             Human engineering fcr applied natural language processing
                                                            ...                                                        
95                                                           Similarity based approaches to natural language processing
96                                                  Opportunities for natural language processing research in education
97            Sentiment analyzer Extract

In [9]:
def words(text):
    from spacy.matcher import Matcher
    word_ = []
    doc = nlp(text)
    patterns = [[{'LOWER':'new'},
                 {'POS':'NOUN'}],
              [{'LOWER':'lexical'},
              {'POS':'NOUN'}],
              [{'LOWER':'pattern'},
              {'POS':'NOUN'}],
              [{'LOWER':'system'},
              {'POS':'NOUN'}],
              [{'LOWER':'using'},
              {'POS':'NOUN'}]]
    matcher = Matcher(nlp.vocab)
    for pattern in patterns:
        matcher.add("matching", None, pattern)
        matches = matcher(doc)
        for i in range(0,len(matches)):
            token = doc[matches[i][1]:matches[i][2]]
            word_.append(str(token))
        for word in word_:
            print(word)
    return word_

df.titles.apply(words).values

Lexical knowledge
Lexical knowledge
Lexical knowledge
Lexical knowledge
Lexical knowledge
Lexical knowledge
Lexical knowledge
Lexical knowledge
Lexical knowledge
Lexical knowledge
system bias
system bias
system bias
new approach
new approach
new approach
new approach
new approach
new approach
new approach
new approach
new approach
new approach
new approach
new approach
new approach
new approach
new approach


array([list([]), list([]), list([]), list([]), list([]), list([]),
       list([]), list([]), list([]), list([]), list([]), list([]),
       list([]), list([]), list([]), list([]), list([]), list([]),
       list([]), list([]), list([]), list([]), list([]), list([]),
       list([]), list([]), list([]), list([]), list([]), list([]),
       list([]), list([]), list([]), list([]), list([]), list([]),
       list([]), list([]), list([]), list([]), list([]), list([]),
       list([]), list([]), list([]), list([]), list([]), list([]),
       list([]), list([]), list([]), list([]), list([]), list([]),
       list([]), list([]), list([]), list([]), list([]), list([]),
       list([]), list([]), list([]), list([]), list([]), list([]),
       list([]), list([]), list([]),
       list(['Lexical knowledge', 'Lexical knowledge', 'Lexical knowledge', 'Lexical knowledge']),
       list([]), list([]), list([]), list([]), list([]), list([]),
       list([]), list([]), list([]), list([]), list([]), lis

In [10]:
def pattern(text):
    from spacy.matcher import Matcher
    patterns_ = []
    doc = nlp(text)
    prog_list = ['method', 'approach','analysis',
                 'techniques','research']
    patterns = [[{'LOWER':{'IN':prog_list},'OP':'+'}],
               [{'POS': 'NOUN'}, {'LOWER': 'of'}, {'POS':'NOUN'}],
               [{'POS': 'VERB'}, {'LOWER': 'for'}, {'POS':'NOUN'}]]
    matcher = Matcher(nlp.vocab) 
    for pattern in patterns:
        matcher.add("matching", None, pattern) 
        matches = matcher(doc)
        for i in range(0,len(matches)):
            start, end = matches[i][1], matches[i][2]
            if doc[start].pos_=='DET':
                start = start+1
            span = str(doc[start:end])
            if (len(patterns_)!=0) and (patterns_[-1] in span):
                patterns_[-1] = span
            else:
                patterns_.append(span)
    return patterns_

df.titles.apply(pattern).values

array([list([]), list([]), list([]), list([]), list([]), list([]),
       list([]), list([]), list([]), list([]), list(['approach']),
       list([]), list([]), list([]), list([]), list([]), list([]),
       list([]), list([]), list([]), list([]), list([]), list([]),
       list([]), list([]), list([]), list([]), list([]), list([]),
       list([]), list([]), list([]), list([]), list(['analysis']),
       list([]), list([]), list([]), list([]), list(['approach']),
       list([]), list([]), list([]), list(['research']), list([]),
       list([]), list([]), list([]), list([]), list([]), list([]),
       list([]), list([]), list([]), list([]), list([]), list([]),
       list(['ontology of space']), list([]), list(['analysis']),
       list([]), list([]), list([]), list([]), list([]), list([]),
       list([]), list(['processing of lyrics']), list([]), list([]),
       list([]), list([]), list(['techniques']), list([]), list([]),
       list([]), list([]), list([]), list([]), list([]), li

## **2. Domain-specific information extraction (10 points)**

For the legal case used in the data cleaning exercise: [01-05-1 Adams v Tanner.txt](https://raw.githubusercontent.com/unt-iialab/info5731_spring2021/main/class_exercises/01-05-1%20%20Adams%20v%20Tanner.txt), use [legalNLP](https://lexpredict-lexnlp.readthedocs.io/en/latest/modules/extract/extract.html#nlp-based-extraction-methods) to extract the following inforation from the text (if the information is not exist, just print None):

(1) acts, e.g., “section 1 of the Advancing Hope Act, 1986”

(2) amounts, e.g., “ten pounds” or “5.8 megawatts”

(3) citations, e.g., “10 U.S. 100” or “1998 S. Ct. 1”

(4) companies, e.g., “Lexpredict LLC”

(5) conditions, e.g., “subject to …” or “unless and until …”

(6) constraints, e.g., “no more than”

(7) copyright, e.g., “(C) Copyright 2000 Acme”

(8) courts, e.g., “Supreme Court of New York”

(9) CUSIP, e.g., “392690QT3”

(10) dates, e.g., “June 1, 2017” or “2018-01-01”

(11) definitions, e.g., “Term shall mean …”

(12) distances, e.g., “fifteen miles”

(13) durations, e.g., “ten years” or “thirty days”

(14) geographic and geopolitical entities, e.g., “New York” or “Norway”

(15) money and currency usages, e.g., “$5” or “10 Euro”

(16) percents and rates, e.g., “10%” or “50 bps”

(17) PII, e.g., “212-212-2121” or “999-999-9999”

(18) ratios, e.g.,” 3:1” or “four to three”

(19) regulations, e.g., “32 CFR 170”

(20) trademarks, e.g., “MyApp (TM)”

(21) URLs, e.g., “http://acme.com/”

(22) addresses, e.g., “1999 Mount Read Blvd, Rochester, NY, USA, 14615”

(23) persons, e.g., “John Doe

In [5]:
!pip install lexnlp==0.2.7
import nltk
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('punkt')

     |████████████████████████████████| 8.0MB 6.4MB/s 
     |████████████████████████████████| 22.2MB 1.9MB/s 
     |████████████████████████████████| 8.8MB 23.1MB/s 
     |████████████████████████████████| 10.3MB 27.5MB/s 
     |████████████████████████████████| 614kB 46.4MB/s 
     |████████████████████████████████| 61kB 7.0MB/s 
     |████████████████████████████████| 15.2MB 190kB/s 
     |████████████████████████████████| 9.5MB 30.4MB/s 
     |████████████████████████████████| 61kB 6.1MB/s 
     |████████████████████████████████| 358kB 41.3MB/s 
     |████████████████████████████████| 1.2MB 43.0MB/s 
     |████████████████████████████████| 61kB 6.0MB/s 
     |████████████████████████████████| 235kB 55.2MB/s 
     |████████████████████████████████| 133kB 56.1MB/s 
     |████████████████████████████████| 61kB 6.7MB/s 
  Created wheel for gensim: filename=gensim-3.4.0-cp37-cp37m-linux_x86_64.whl size=23316475 sha256=e2084b626c8837be750f5f0f94a4157a790a64856da569babc64f71d74233cec
  St

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [6]:
# write your code here
import lexnlp
import re
import lexnlp.extract.en.acts

with open('01-05-1 Adams v Tanner.txt', 'r', encoding="utf8") as file:
    text = file.read()
text = re.sub(r'\n',' ', str(text))
#acts
lexnlp.extract.en.acts.get_act_list(text)

[]

In [7]:
# amounts
import lexnlp.extract.en.amounts
for amount in list(lexnlp.extract.en.amounts.get_amounts(text)):
    print(amount, end=', ')

5.0, 740.0, 1843.0, 2.0, 1.0, 4.0, 2.0, 1821.0, 5.0, 1.0, 1840.0, 3777.0, 80.0, 100.0, 30.0, 1839.0, 741.0, 22.0, 1840.0, 14000.0, 120.0, 1.0, 1840.0, 3.0, 4.0, 1.0, 1.0, 1840.0, 2.0, 1.0, 361.0, 1.0, 307.0, 6.0, 604.0, 1.0, 2.0, 418.0, 422.0, 7.0, 34.0, 41.0, 167.0, 742.0, 3.0, 112.0, 207.0, 3.0, 338.0, 424.0, 5.0, 26.0, 13.0, 235.0, 8.0, 693.0, 4.0, 1821.0, 167.0, 2.0, 2.0, 216.0, 3.0, 66.0, 4.0, 130.0, 29.0, 2.0, 241.0, 2.0, 332.0, 2.0, 422.0, 9.0, 112.0, 743.0, 9.0, 39.0, 14000.0, 1840.0, 744.0, 5.0, 182.0, 3.0, 368.0, 1.0, 397.0, 6.0, 604.0, 1.0, 1821.0, 167.0, 745.0, 4.0, 746.0, 4.0, 210.0, 46.0, 747.0, 5.0, 5.0, 740.0, 1843.0, 284.0, 2019.0, 9.0, 1.0, 55.0, 266.0, 271.0, 1876.0, 2.0, 47.0, 362.0, 376.0, 1872.0, 3.0, 45.0, 329.0, 334.0, 1871.0, 4.0, 31.0, 526.0, 527.0, 1858.0, 5.0, 21.0, 333.0, 335.0, 1852.0, 6.0, 8.0, 145.0, 147.0, 1857.0, 7.0, 65.0, 256.0, 258.0, 3.0, 1880.0, 8.0, 4.0, 913.0, 914.0, 1887.0, 9.0, 103.0, 464.0, 1936.0, 3.0, 1.0, 9.0, 39.0, 1828.0, 2.0, 2.0, 5.0, 

In [8]:
import lexnlp.extract.en.citations
_ = [print(citation) for citation in list(lexnlp.extract.en.citations.get_citations(text))]

(5, 'Ala.', 'Alabama Reports', 740, None, None, None)
(5, 'Ala.', 'Alabama Reports', 740, '1843', None, None)
(55, 'Ala.', 'Alabama Reports', 266, '271', None, None)
(47, 'Ala.', 'Alabama Reports', 362, '376', None, None)
(45, 'Ala.', 'Alabama Reports', 329, '334', None, None)
(31, 'Ala.', 'Alabama Reports', 526, '527', None, None)
(21, 'Ala.', 'Alabama Reports', 333, '335', None, None)
(8, 'Cal.', 'California Reports', 145, '147', None, None)
(65, 'Ala.', 'Alabama Reports', 256, '258', None, None)
(4, 'S.W.', 'South Western Reporter', 913, '914', None, None)
(103, 'A.L.R.', 'American Law Reports', 464, None, None, None)
(9, 'Cow.', "Cowen's Reports", 39, None, None, None)
(5, 'Port.', 'Alabama Reports, Porter', 182, None, None, None)
(9, 'Johns.', "Johnson's Reports", 108, None, None, None)


In [9]:
# companies
import lexnlp.extract.en.entities.nltk_re
_ = [print(company) for company in list(lexnlp.extract.en.entities.nltk_re.get_companies(text))]

Lehman, Durr Co, (17983, 18001)


In [10]:

# conditions
import lexnlp.extract.en.conditions
_ = [print(condition) for condition in list(lexnlp.extract.en.conditions.get_conditions(text))]

('until', '4 Cases that cite this headnote  [2] Creditors’ Remedies Lien and Priority Under St.1821, prohibiting a levy on a crop', '')
('until', 'on a growing crop, nor does such lien attach', '')
('if', 'It was proved by the claimants, by the production of a written contract, that Harrison, on the twenty-second of May, 1840, in consideration that the claimants were involved, as indorsers for Burton & Harrison of Sumter county, and were then exposed to an execution, amounting to upwards of fourteen thousand dollars, bargained and sold to the claimants all his growing crop of cotton &c., consisting of one hundred and twenty acres, &c. Allen Harrison promised and obliged himself to give up his crop to the use of the claimants at any time to save them from suffering as his indorsers;', '')
('when', 'The claimants came from Tennessee, (where they resided) about the first of September, 1840, bringing with them three or four white laborers, and took possession of the crop and slaves, and wi

In [11]:
# constraints
import lexnlp.extract.en.constraints
_ = [print(constraint) for constraint in list(lexnlp.extract.en.constraints.get_constraints(text))]

('after', 'on a growing crop, nor does such lien attach until', '')
('after', '', ' and that alias and pluries fieri facias’, issued regularly up to the time levy was made; that the cotton levied on was growed on the plantation of harrison, and cultivated by the hands in his service.')
('first of', 'the claimants came from tennessee, (where they resided) about the', '')
('first of', 'the court charged the jury, that the plaintiff had no lien by virtue of his judgment, and execution on the growing crop; that harrison had a right to convey it, without being in any manner restrained by them; that the writing adduced, was a sale of the crop, but if it was not, and the lien of the fieri facias would have attached upon it, when gathered, yet if the claimants obtained possession on the', '')
('after', 'it merely inhibits the levy, but the lien attaches, and a levy and sale may be made', '')
('more than', 'taking this to be clear *744 law, and it will be seen, that the defendant in execution a

In [12]:
# copyrights
import lexnlp.extract.en.copyright
_ = [print(copyright) for copyright in list(lexnlp.extract.en.copyright.get_copyright(text))]

('©', '2019', 'Thomson Reuters. No')


In [29]:
#courts
import pandas
import lexnlp.extract.en.courts
import lexnlp.extract.en.dict_entities

court_df = pandas.read_csv("https://raw.githubusercontent.com/LexPredict/lexpredict-legal-dictionary/1.0.5/en/legal/us_courts.csv")
# Create config objects
court_config_data = [lexnlp.extract.en.dict_entities.entity_config(0, "Eastern District of Virginia", 0, ["E.D. Va."]),
    lexnlp.extract.en.dict_entities.entity_config(1, "Western District of Virginia", 0, ["W.D. Va."])]
for entity, alias in lexnlp.extract.en.courts.get_courts(text, court_config_data):
  print("entity=", entity)
  print("alias=", alias)
entity= (98, 'Eastern District of Virginia', 0, [('Eastern District of Virginia', None, False, None), ('E.D. Va.', None, False, None)])
alias= ('E.D. Va.', None, False, None)
entity= (70, 'Southern District of New York', 0, [('Southern District of New York', None, False, None), ('S.D.N.Y.', None, False, None)])
alias= ('S.D.N.Y.', None, False, None)

AttributeError: ignored

In [18]:
# CUSIP
import lexnlp.extract.en.cusip
list(lexnlp.extract.en.cusip.get_cusip(text))

[]

In [19]:
# dates
import lexnlp.extract.en.dates
_ = [print(date) for date in list(lexnlp.extract.en.dates.get_dates(text))]

2021-06-01
1840-11-01
1839-10-01
1840-09-01
1840-05-01
1840-05-01
2021-12-01
2021-12-01
2021-01-01
2021-01-01
2021-01-01
2021-03-21
2021-06-01
2021-07-01
2021-11-01


In [20]:
# definitions
import lexnlp.extract.en.definitions
list(lexnlp.extract.en.definitions.get_definitions(text))

[]

In [21]:
# distances
import lexnlp.extract.en.distances
list(lexnlp.extract.en.distances.get_distances(text))

[]

In [22]:
#durations
import lexnlp.extract.en.durations
list(lexnlp.extract.en.durations.get_durations(text))

[('second', Decimal('20.0'), Decimal('0.0002')),
 ('year', Decimal('6.0'), Decimal('2190.0'))]

In [27]:
# geoentities
import lexnlp.extract.en.geoentities
lexnlp.extract.en.geoentities.get_geoentities(text)

TypeError: ignored

In [30]:
#money
import lexnlp.extract.en.money
list(lexnlp.extract.en.money.get_money(text))

[(Decimal('100.0'), 'USD'),
 (Decimal('14000.0'), 'USD'),
 (Decimal('14000.0'), 'USD')]

In [31]:
#percents
import lexnlp.extract.en.percents
list(lexnlp.extract.en.percents.get_percents(text))

[]

In [32]:
#pii 
import lexnlp.extract.en.pii
list(lexnlp.extract.en.pii.get_pii(text))

[]

In [33]:
#ratios
import lexnlp.extract.en.ratios
list(lexnlp.extract.en.ratios.get_ratios(text))

[]

In [34]:
#regulations
import lexnlp.extract.en.regulations
list(lexnlp.extract.en.regulations.get_regulations(text))

[]

In [35]:
# trademarks
import lexnlp.extract.en.trademarks
list(lexnlp.extract.en.trademarks.get_trademarks(text))

[]

In [36]:
#urls
import lexnlp.extract.en.urls
list(lexnlp.extract.en.urls.get_urls(text))

[]